# 'Omgeving Leiden' tag verhuizen en opschonen

Het probleem is nu dat 
 1. `Omgeving Leiden` children heeft die verwijzen naar `item` gids en niet naar `tag` gids. 
 1. `Omgeving Leiden` op  een verkeerde plek in de boom staat


In [ ]:
%run ../__init__edwh__notebooks.ipynb

# Probleem 1: .children
Laten we beginnen met de Tag zelf. 

In [ ]:
tag = Tag("Omgeving Leiden")

De `.children` waarnaar deze tag verwijst zijn gids uit de `item` tabel, en niet de `tag` tabel. Dat heb ik onjuist ingevoerd.  
Eerst controleren we dat de gegevens inderdaad onjuist zijn:

In [ ]:
tag_children = tag.row.children
len(tag_children)

In [ ]:
# handig.names([Tag(_) for _ in tag_children])

Hierboven zie je dat er geen tag voorkomt met dit gid.  
Laten we de item tabel controleren: 

In [ ]:
item_rows = db(db.item.gid.belongs(tag_children)).select(db.item.gid, db.item.name)
for item_row in item_rows: 
    print(item_row)
for child_tag in tag_children: 
    print(child_tag)
    if child_tag not in [_.gid for _ in item_rows]:
        # als dit niet geprint wordt zijn er geen tags die voorkomen, het zijn allemaal item tags. 
        print('echte child tag:',child_tag)

Hierboven zie je dat de `c534a1c2-6386-410d-9fad-18ff3a46542c` tag 2 keer voorkomt, zodat van de 11 gids, er 10 unieke zijn, en die 10 unieke gids komen dus voor in de items.   
Wat betekend dat **alle** children van `Omgeving Leiden` items zijn, die hier niet in horen. Die mogen allemaal weg. 

In [ ]:
omgeving_leiden = Tag("Omgeving Leiden")
print('Voor de clear: Children van omgeving leiden:',omgeving_leiden.row.children)
omgeving_leiden.row.update_record(children=[])
print('Na de clear: Children van omgeving leiden:',omgeving_leiden.row.children)
Tag.refresh(Tag.db)
# herlaad zodat children leeg is, en de meest actuele versie van de database komt 
omgeving_leiden = Tag("Omgeving Leiden")

# Probleem 2: Verkeerde plek in de boom

## Er komt een nieuwe root node bij
Omdat er meerdere omgevingsprojecten zijn (zoals leiden en zeeland) is het aannemelijk dat er meer bij komen. 

Daarom moet er een nieuwe rootnode bij komen "Omgevingen" waar een tag voor zeeland en bijvoorbeeld "Omgeving Leiden" bijkomen. 

Dus: 
 1. nieuwe tag `Omgevingen` zonder parent met metadata `System`
 1. nieuwe tag `Omgeving Zeeland` met als parent `Omgevingen` en metadata `Item` (want het moet aan een item gekoppeld kunnen worden).  
 1. aanpassing aan `Omgeving Leiden` met als parent `Omgevingen` en metadata `Item` (want het moet aan een item gekoppeld kunnen worden).  
 1. op zoek naar alle items die eigenlijk bij Leiden horen (via school tags) en deze koppelen aan `Omgeving Leiden` als ze niet al gekoppeld zijn.  
    Deze laatste bewerking valt onder een ander notebook, omdat dit regulatig uitgevoerd moet worden. 
   
 

In [ ]:
print('tag.parents:',handig.names(omgeving_leiden.parents()))
print('tag.metatags:',handig.names(omgeving_leiden.metatags()))

In [ ]:
def _print(tag, parent, depth):
    if tag == parent:
        return False
    print('   ' * depth, tag.name, handig.names(tag.metatags()))
    if tag.name == 'User generated tags':
        print('   ' * (depth + 1), ' *** hidden ***')
    return tag.name != 'User generated tags'
# 'parents' richting
tag.walk(_print, direction='parents')

###  1. nieuwe tag `Omgevingen` zonder parent met metadata `System`

In [ ]:
try:
    TOmgevingen = Tag.new("Omgevingen","Parent node voor omgeving tags.", meta_tags=[TSystem])
except:
    TOmgevingen = Tag("Omgevingen")
TOmgevingen.walk(_print)

In [ ]:
db.commit()

###  2. nieuwe tag `Omgeving Zeeland` met als parent `Omgevingen` en metadata `Item`

In [32]:
try:
    TZeeland = Tag.new("Omgeving Zeeland","Ervaringen uit Zeeland", meta_tags=[TSystem], parents=[TOmgevingen])
except:
    pass 
Tag.refresh(Tag.db)
TZeeland = Tag("Omgeving Zeeland")
TZeeland.walk(_print)
db.commit()

 Omgeving Zeeland ['System']


In [33]:
%%sql 
select * from tag where name = 'Omgeving Zeeland' order by id desc limit 1 

 * postgresql://postgres@pgpool:5432/backend
1 rows affected.


id,platform,gid,name,slug,parents,description,meta_tags,children
32611,None,0b1b6d60-89e2-447d-8ce8-0ea257af8b71,Omgeving Zeeland,omgeving-zeeland,|c9e797f7-be49-44f6-a6ca-45dafb912022|,Ervaringen uit Zeeland,|19682a99-50a3-4fc0-bb67-e0f6eff5da55|,||


###  3. aanpassing aan `Omgeving Leiden` met als parent `Omgevingen` en metadata `Item`

In [ ]:
TOmgevingLeiden = Tag('Omgeving Leiden')
TOmgevingLeiden.row.update_record(parents=[TOmgevingen.gid], meta_tags=[TItem.gid])
db.commit()
Tag.refresh(Tag.db)
TOmgevingLeiden = Tag('Omgeving Leiden')
TOmgevingLeiden
TOmgevingLeiden.walk(_print, direction='parents')